### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import os

# File to Load (Remember to Change These)
school_data_to_load = os.path.join("Resources","schools_complete.csv")
student_data_to_load = os.path.join("Resources","students_complete.csv")

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_data_complete['pass_math'] = school_data_complete['math_score'].apply(lambda x : 1 if x>=70 else 0)
school_data_complete['pass_read'] = school_data_complete['reading_score'].apply(lambda x : 1 if x>=70 else 0)

school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,pass_math,pass_read
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,1,0
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,0,1
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,0,1
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,0,0
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,1,1


In [2]:
#convert int to rounded string with following %
def percent(x):
    return str(round(x,2))+'%'

#take a dataframe, and groupby column name
#return new dataframe with Average Math Score, Average Reading Score, % Passing Math, % Passing Reading, % Overall Passing Rate
def scores_by(df,cut, bins, bin_names):
    new_df = df.assign(Bin=lambda x:pd.cut(schools[cut],bins=bins,labels=bin_names,right=False))\
        .groupby('Bin')\
        .agg({'Total Students':'sum',
              'math_score':'sum',
              'reading_score':'sum',
              'pass_math':'sum',
              'pass_read':'sum'
             })
    return new_df

#take a dataframe and return the avg math/reading scores, % passing math/reading, and % overall passing rate
def scores_math(df):
    df['Average Math Score'] = df['math_score']/df['Total Students']
    df['Average Reading Score'] = df['reading_score']/df['Total Students']
    df['% Passing Math'] = (df['pass_math']/df['Total Students'])*100
    df['% Passing Reading'] = (df['pass_read']/df['Total Students'])*100
    df['% Overall Passing Rate'] = (df['% Passing Math']+df['% Passing Reading'])/2

    return df

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
pass_math = (school_data_complete['pass_math'].sum()/len(student_data)*100)
pass_read = (school_data_complete['pass_read'].sum()/len(student_data)*100)

district_df = pd.DataFrame({
    'Total Schools':[len(school_data)],
    'Total Students':[len(student_data)],
    'Total Budget':[school_data['budget'].sum()],
    'Average Math Score':[student_data['math_score'].mean()],
    'Average Reading Score':[student_data['reading_score'].mean()],
    '% Passing Math':[pass_math],
    '% Passing Reading':[pass_read],
    '% Overall Passing Rate':[(pass_math+pass_read)/2]
})
district_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.393158


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [4]:
schools = school_data_complete.groupby(['school_name','type','budget'])\
    .agg({
        'Student ID':'count',
        'math_score':'sum',
        'reading_score':'sum',
        'pass_math':'sum',
        'pass_read':'sum'
    })\
    .reset_index(['type','budget'])\
    .rename(columns={'type':'School Type',
                     'budget':'Total School Budget',
                     'Student ID':'Total Students',
                    })

schools = scores_math(schools)

schools['Per Student Budget'] = (schools['Total School Budget']/schools['Total Students'])*100

columns = ['School Type','Total Students','Total School Budget','Per Student Budget',
           'Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate']

schools.sort_values('% Overall Passing Rate', ascending=False)[columns].head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,58200.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,1043130,63800.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,585858,60900.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,917500,62500.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574,57800.0,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [5]:
schools.sort_values('% Overall Passing Rate', ascending=True)[columns].head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363,63700.0,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,1884411,63900.0,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,1910635,65500.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,65000.0,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,1763916,64400.0,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
grade = school_data_complete.groupby(['grade','school_name'])\
    .agg({'math_score':'mean',
          'reading_score':'mean'
    })

cols = school_data_complete['grade'].unique()

grade.unstack(level='grade')['math_score'][cols]

grade,9th,12th,11th,10th
school_name,,,,
Bailey High School,77.083676,76.492218,77.515588,76.996772
Cabrera High School,83.094697,83.277487,82.765560,83.154506
Figueroa High School,76.403037,77.151369,76.884344,76.539974
Ford High School,77.361345,76.179963,76.918058,77.672316
Griffin High School,82.044010,83.356164,83.842105,84.229064
Hernandez High School,77.438495,77.186567,77.136029,77.337408
Holden High School,83.787402,82.855422,85.000000,83.429825
Huang High School,77.027251,77.225641,76.446602,75.908735
Johnson High School,77.187857,76.863248,77.491653,76.691117


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
grade.unstack(level='grade')['reading_score'][cols]

grade,9th,12th,11th,10th
school_name,,,,
Bailey High School,81.303155,80.912451,80.945643,80.907183
Cabrera High School,83.676136,84.287958,83.788382,84.253219
Figueroa High School,81.198598,81.384863,80.640339,81.408912
Ford High School,80.632653,80.662338,80.403642,81.262712
Griffin High School,83.369193,84.013699,84.288089,83.706897
Hernandez High School,80.866860,80.857143,81.396140,80.660147
Holden High School,83.677165,84.698795,83.815534,83.324561
Huang High School,81.290284,80.305983,81.417476,81.512386
Johnson High School,81.260714,81.227564,80.616027,80.773431


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [8]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 58500, 61500, 64500, 67500]
spend_names = ["<$585", "$585-615", "$615-645", "$645-675"]
columns = ['Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate']

In [9]:
spend = scores_by(schools,'Per Student Budget',spending_bins,spend_names)
spend = scores_math(spend)\
                .rename_axis('Spending Ranges (Per Student)')
spend[columns]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.363065,83.964039,93.702889,96.686558,95.194724
$585-615,83.529196,83.838414,94.124128,95.886889,95.005509
$615-645,78.061635,81.434088,71.400428,83.614770,77.507599
$645-675,77.049297,81.005604,66.230813,81.109397,73.670105


## Scores by School Size

* Perform the same operations as above, based on school size.

In [10]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
size_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [11]:
size = scores_by(schools,'Total Students',size_bins,size_names,)
size = scores_math(size)\
        .rename_axis('School Size')
size[columns]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,94.996400
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,95.194790
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,75.388769


## Scores by School Type

* Perform the same operations as above, based on school type.

In [12]:
types = schools.groupby('School Type')\
        .agg({'Total Students':'sum',
              'math_score':'sum',
              'reading_score':'sum',
              'pass_math':'sum',
              'pass_read':'sum'
             })
types = scores_math(types)
types[columns]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818
